In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase import firebase
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib
import urllib.request as ur
import sys

In [7]:
model = tf.keras.models.load_model('/home/vh/Documents/Plant-Disease-Website/model/saved_model', compile=False)

2023-05-10 21:23:28.406486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
# cred = credentials.Certificate('/home/vh/Documents/serviceAccount.json')

# app = firebase_admin.initialize_app(cred)

In [17]:
db = firestore.client()

images_ref = db.collection(u'images')

query = images_ref.where(u'status', u'==', u'')

docs = query.stream()

urls = []

for doc in docs:
    doc_dict = doc.to_dict()

    imageUrl = doc_dict['imageUrl']
    
    print(imageUrl)
    
    urls.append(imageUrl)

print("Num of images: ",len(urls))

https://firebasestorage.googleapis.com/v0/b/plant-pests-detection.appspot.com/o/images%2Fphoto5941516755127936231.jpg?alt=media&token=ce6303d3-1aa9-4aef-a73e-3e1a26c84e34
Num of images:  1


In [18]:
images = []
for i in range(0,len(urls)):
    image = "image_" + str(i) +".jpg"
    images.append(image)
    urllib.request.urlretrieve(urls[i], image)
    with PIL.Image.open(image) as img:
        img = img.resize((256,256))
        img.save(f'./{image}')

In [19]:
print(images)
test = pd.DataFrame({'image':images})
print(test)

['image_0.jpg']
         image
0  image_0.jpg


In [20]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )
test_data = datagen.flow_from_dataframe(
    test,
    directory = '.',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

print(test_data)

preds = model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

Found 1 validated image filenames.
1/1 [==============================] - 0s 198ms/step
[[0.02997818 0.31402257 0.00727026 0.00333117 0.45867714 0.08087564]]
[[1, 4]]


2023-05-10 21:29:51.628920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [21]:
labels = {'complex': 0, 'frog_eye_leaf_spot': 1, 'healthy': 2, 'powdery_mildew': 3, 'rust': 4, 'scab': 5}
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []
temp = []
for i in indices:
    i = [i[0]]
    temp.append(i)
indices = temp

for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

{0: 'complex', 1: 'frog_eye_leaf_spot', 2: 'healthy', 3: 'powdery_mildew', 4: 'rust', 5: 'scab'}
['frog_eye_leaf_spot']


In [22]:
# cred = credentials.Certificate('/home/vh/Documents/serviceAccount.json')

# firebase_admin.initialize_app(cred)

# Initialize Firestore client

db = firestore.client()

firebase = firebase.FirebaseApplication('https://plant-pests-detection-default-rtdb.firebaseio.com/', None)

images_ref = db.collection(u'images')

# Create a query to filter documents by status equal to an empty string
query = images_ref.where(u'status', u'==', u'')

# Get the documents that match the query
docs = query.stream()

index = 0

for doc in docs:
    result = testlabels[index]
    
    detect_ref = db.collection(u'images').document(doc.id)

    detect_ref.update({u'status': result})

    firebase.put('status', 'status', result)
    
    index += 1

/home/vh/.local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
